In [4]:
import pandas as pd 
from scrape import get_fox_table 
import plotly.graph_objs as go 
import plotly.plotly as py 
import numpy as np
import datetime

# Merging STANDARD & DISCIPLINE tables.. so I can subtract penalties from goal count
df = pd.read_csv('liga_attack.csv',header =0).merge(
     pd.read_csv('liga_discipline.csv',header=0)[['Name','Penalty Kick Goals']],
     on='Name',copy=False) 

df['NPG'] = df['Goals']-df['Penalty Kick Goals'] # NPG is Non-Penalty Goals 
df['ShotConversion'] = df['Goals']/df['Shots On Goal'] # At what rate do on-target shots become goals?  
df['ShotPercentage'] = df['Shots On Goal']/df['Shots'] # At what rate are shots put on target?  

# Players with greater than or equal to twenty shots, big enough sample size
enough_shots = df.loc[(df['Shots']>=20),]

# Extract desired columns
plot_data = enough_shots[['Name','TEAM CODE','Shots','Shots On Goal','ShotConversion','ShotPercentage','NPG']]


# Importing League Positions
tables = pd.read_csv("liga_table.csv",header = 0)
print(tables)
#curr_table = tables.loc[tables.matchday==tables.matchday.max(),

# Use Team Codes in "tables" (containing team position) to map position to plot_data
plot_data['Position']=plot_data['TEAM CODE'].map(dict(zip(tables['TEAM CODE'].tolist(),tables['position'].tolist())))


# Assigning colors to data points - DIV here means "division" in the table.. Top 4, Mid Table... etc
plot_data = plot_data.assign(COL = 'green',DIV='1-4')
plot_data.loc[plot_data.Position > 4,['COL','DIV']]=['#ffe700','5-11']
plot_data.loc[plot_data.Position > 11,['COL','DIV']]=['orange','12-17']
plot_data.loc[plot_data.Position > 17,['COL','DIV']]=['red','18-20']
                                              
                                              
# Stores data for plotly.iplot
traces=[]

# Append traces by , so the plotly legend allows one to filter by table division
for div, div_table in plot_data.groupby('DIV',sort=False): # Sort = False so iterables are not sorted
    traces.append(go.Scatter(
        x = div_table['ShotPercentage'], 
        y = div_table['ShotConversion'],
        text = div_table['Name'].str.split(' ').str[-1], # Get Last Name
        textposition = 'bottom', # Place text under data point
        name=div, # Legend name of divison is division itself
        textfont=dict(size=10,color=div_table.COL), # Dictionary containing text label properties  
        hovertext = div_table.Name + ' (' + div_table['TEAM CODE']+ ')' + '<br>' + 
        'Goals: '+ div_table['NPG'].astype(str)+
        ' - Shots: '+div_table['Shots'].astype(str)+'('+ div_table['Shots On Goal'].astype(str)+')', # Hover Label properties
        hoverinfo='text', # What should the hover display? Hovertext
        marker= dict(size = div_table['NPG']*5, 
                     color = div_table.COL,
                    line = dict(
                        color = div_table.COL
                    )), # Marker properties
        mode = 'markers+text'
    ))
    
    # Graph Layout Properties
    layout = dict(title = ' Shot Statistics - Forwards (>20 Shots) <br> {} :: @FootyNumb3rs'.format(datetime.date.today()),
                  titlefont=dict(size=15),
                  xaxis = dict(title = 'Shots on Goal / Shots'),
                  yaxis = dict(title = 'Goals / Shots on Goal'),
                  height = 600, width = 800,
                  showlegend=True,
                  hovermode='closest',
                  images=[dict(
                      source="https://upload.wikimedia.org/wikipedia/commons/thumb/1/13/LaLiga.svg/2000px-LaLiga.svg.png",
                      xref="paper", yref="paper",
                      x=.18, y=1.06,
                      sizex=0.21, sizey=0.21,
                      xanchor="right", yanchor="bottom"
                  )])     

fig = go.Figure(data = traces,layout=layout)
py.iplot(fig,filename='liga-attack2')


    position                 teamName  \
0          1             FC Barcelona   
1          2  Club Atlético de Madrid   
2          3              Valencia CF   
3          4           Real Madrid CF   
4          5               Sevilla FC   
5          6            Villarreal CF   
6          7                 SD Eibar   
7          8                Getafe CF   
8          9  Real Sociedad de Fútbol   
9         10                Girona FC   
10        11            Athletic Club   
11        12               CD Leganes   
12        13               Real Betis   
13        14             RCD Espanyol   
14        15         RC Celta de Vigo   
15        16               Levante UD   
16        17         Deportivo Alavés   
17        18   RC Deportivo La Coruna   
18        19                Málaga CF   
19        20            UD Las Palmas   

                                             crestURI TEAM CODE  playedGames  \
0   http://upload.wikimedia.org/wikipedia/de/a/aa/...     

c:\users\alice\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

